In [1]:
// Auto-generated, ignore.
import "../../../../scripts/create_global_node_vars.js"

[Module: null prototype] {  }

# OpenAI Assistant

:::info

The [OpenAI Assistant API](https://platform.openai.com/docs/assistants/overview) is still in beta.

:::

OpenAI released a new API for a conversational agent like system called Assistant.

You can interact with OpenAI Assistants using OpenAI tools or custom tools. When using exclusively OpenAI tools, you can just invoke the assistant directly and get final answers. When using custom tools, you can run the assistant and tool execution loop using the built-in `AgentExecutor` or write your own executor.
OpenAI assistants currently have access to two tools hosted by OpenAI: [code interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter), and [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval).

We've implemented the assistant API in LangChain with some helpful abstractions. In this guide we'll go over those, and show how to use them to create powerful assistants.


## Creating an assistant

Creating an assistant is easy. Use the `createAssistant` method and pass in a model ID, and optionally more parameters to further customize your assistant.

In [1]:
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";

const assistant = await OpenAIAssistantRunnable.createAssistant({
  model: "gpt-4-1106-preview",
});
await assistant.invoke({
  content: "Hello world!",
});

[
  {
    id: "msg_5WRVMVgciA8v3ldUEPo1STxG",
    object: "thread.message",
    created_at: 1704228199,
    thread_id: "thread_jpuo2hk8ollfIAxsp53AoZU9",
    role: "assistant",
    content: [
      {
        type: "text",
        text: {
          value: "Hello! How can I assist you today? If you have any questions or need information, feel free to ask.",
          annotations: []
        }
      }
    ],
    file_ids: [],
    assistant_id: "asst_kVic7w3NMi80MHkOUl07qMuC",
    run_id: "run_t0aAJUuALkiZH48krSUbBpbk",
    metadata: {}
  }
]

If you run into an apiKey error, you can try to pass it directly as clientOptions:

In [6]:
const assistant = await OpenAIAssistantRunnable.createAssistant({
  clientOptions: { apiKey: process.env.OPENAI_API_KEY as string },
  model: "gpt-4-1106-preview",
});

If you have an existing assistant, you can pass it directly into the constructor:

In [ ]:
const assistant = new OpenAIAssistantRunnable({
  assistantId: "asst_RtW03Vs6laTwqSSMCQpVND7i",  // Replace with your assistant ID
  // asAgent: true
});

In this next example we'll show how you can turn your assistant into an agent.

## Assistant as an agent

In [17]:
import { AgentExecutor } from "langchain/agents";
import { StructuredTool } from "langchain/tools";
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";
import * as z from "zod";

The first step is to define a list of tools you want to pass to your assistant.
Here we'll only define one for simplicity's sake, however the assistant API allows for passing in a list of tools, and from there the model can use multiple tools at once.
Read more about the run steps lifecycle [here](https://platform.openai.com/docs/assistants/how-it-works/runs-and-run-steps)

:::note
Only models released >= 1106 are able to use multiple tools at once. See the full list of OpenAI models [here](https://platform.openai.com/docs/models).
:::

In [18]:
function getCurrentWeather(location: string, _unit = "fahrenheit") {
  if (location.toLowerCase().includes("tokyo")) {
    return JSON.stringify({ location, temperature: "10", unit: "celsius" });
  } else if (location.toLowerCase().includes("san francisco")) {
    return JSON.stringify({ location, temperature: "72", unit: "fahrenheit" });
  } else {
    return JSON.stringify({ location, temperature: "22", unit: "celsius" });
  }
}
class WeatherTool extends StructuredTool {
  schema = z.object({
    location: z.string().describe("The city and state, e.g. San Francisco, CA"),
    unit: z.enum(["celsius", "fahrenheit"]).optional(),
  });

  name = "get_current_weather";

  description = "Get the current weather in a given location";

  constructor() {
    super(...arguments);
  }

  async _call(input: { location: string; unit: string }) {
    const { location, unit } = input;
    const result = getCurrentWeather(location, unit);
    return result;
  }
}
const tools = [new WeatherTool()];

In the above code we've defined three things:

- A function for the agent to call if the model requests it.
- A tool class which we'll pass to the `AgentExecutor`
- The tool list we can use to pass to our `OpenAIAssistantRunnable` and `AgentExecutor`

Next, we construct the `OpenAIAssistantRunnable` and pass it to the `AgentExecutor`.

In [3]:
const agent = await OpenAIAssistantRunnable.createAssistant({
  model: "gpt-3.5-turbo-1106",
  instructions:
    "You are a weather bot. Use the provided functions to answer questions.",
  name: "Weather Assistant",
  tools,
  asAgent: true,
});
const agentExecutor = AgentExecutor.fromAgentAndTools({
  agent,
  tools,
});

Note how we're setting `asAgent` to `true`, this input parameter tells the `OpenAIAssistantRunnable` to return different, agent-acceptable outputs for actions or finished conversations.

Above we're also doing something a little different from the first example by passing in input parameters for `instructions` and `name`.
These are optional parameters, with the instructions being passed as extra context to the model, and the name being used to identify the assistant in the OpenAI dashboard.

Finally to invoke our executor we call the `.invoke` method in the exact same way as we did in the first example.

In [11]:
await agentExecutor.invoke({
  content: "What's the weather in Tokyo and San Francisco?",
});

{
  output: "The current weather in Tokyo is 10°C, and the current weather in San Francisco is 72°F."
}

Here we asked a question which contains two sub questions inside: `What's the weather in Tokyo?` and `What's the weather in San Francisco?`.
In order for the `OpenAIAssistantRunnable` to answer that it returned two sets of function call arguments for each question, demonstrating it's ability to call multiple functions at once.

## Assistant tools

OpenAI currently offers two tools for the assistant API: a [code interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) and a [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) tool.
You can offer these tools to the assistant simply by passing them in as part of the `tools` parameter when creating the assistant.

In [5]:
const assistant = await OpenAIAssistantRunnable.createAssistant({
  model: "gpt-3.5-turbo-1106",
  instructions:
    "You are a helpful assistant that provides answers to math problems.",
  name: "Math Assistant",
  tools: [{ type: "code_interpreter" }],
});

Since we're passing `code_interpreter` as a tool, the assistant will now be able to execute Python code, allowing for more complex tasks normal LLMs are not capable of doing well, like math.

In [12]:
await assistant.invoke({
  content: "What's 10 - 4 raised to the 2.7",
});

[
  {
    id: "msg_gH9tQ6s0CjhbGJqDEjO9FoKN",
    object: "thread.message",
    created_at: 1703625783,
    thread_id: "thread_iDDoAwms3CrL0AZTby0OQs9p",
    role: "assistant",
    content: [
      {
        type: "text",
        text: {
          value: "To solve the expression 10 - 4 raised to the 2.7, you need to follow the order of operations, known "... 544 more characters,
          annotations: []
        }
      }
    ],
    file_ids: [],
    assistant_id: "asst_4SkCTCdLYSelPufZNYQjokes",
    run_id: "run_hlpDCmAYl8IhoDjIBP38jv52",
    metadata: {}
  }
]

Here the assistant was able to utilize the `code_interpreter` tool to calculate the answer to our question.

## Retrieves an assistant

Retrieves an assistant.

In [9]:
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";

const assistant = new OpenAIAssistantRunnable({
  assistantId: "asst_MazQJOnFHWmpNR0MrQUb4Q3c", // Replace with your assistant ID
});
await assistant.getAssistant();

{
  id: "asst_MazQJOnFHWmpNR0MrQUb4Q3c",
  object: "assistant",
  created_at: 1703625650,
  name: "Math Assistant",
  description: null,
  model: "gpt-3.5-turbo-1106",
  instructions: "You are a helpful assistant that provides answers to math problems.",
  tools: [ { type: "code_interpreter" } ],
  file_ids: [],
  metadata: {}
}

## Modifies an assistant

Modifies an assistant.

In [13]:
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";

const assistant = await OpenAIAssistantRunnable.createAssistant({
  name: "Personal Assistant",
  model: "gpt-4-1106-preview",
});
await assistant.modifyAssistant({
  name: "Personal Assistant 2",
});

{
  id: "asst_K9vvk7lhro7VXF6oCJ5ruBlo",
  object: "assistant",
  created_at: 1703625847,
  name: "Personal Assistant 2",
  description: null,
  model: "gpt-4-1106-preview",
  instructions: null,
  tools: [],
  file_ids: [],
  metadata: {}
}

## Delete an assistant

Delete an assistant.

In [14]:
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";

const assistant = await OpenAIAssistantRunnable.createAssistant({
  name: "Personal Assistant",
  model: "gpt-4-1106-preview",
});
await assistant.deleteAssistant();

{
  id: "asst_41KWv1gzsTUMxqdZKtmHMi1b",
  object: "assistant.deleted",
  deleted: true
}

# OpenAI Files

Files are used to upload documents that can be used with features like Assistants and Fine-tuning.

We've implemented the File API in LangChain with create and delete. You can see [the official API reference here](https://platform.openai.com/docs/api-reference/files/object).

The `File` object represents a document that has been uploaded to OpenAI.

```json
{
  "id": "file-abc123",
  "object": "file",
  "bytes": 120000,
  "created_at": 1677610602,
  "filename": "salesOverview.pdf",
  "purpose": "assistants",
}
```



## Create a File

Upload a file that can be used across various endpoints. The size of all the files uploaded by one organization can be up to **100 GB**.

The size of individual files can be a maximum of **512 MB**. See the Assistants Tools guide above to learn more about the types of files supported. The Fine-tuning API only supports `.jsonl` files.


In [32]:
import { OpenAIFiles } from "langchain/experimental/openai_files";
import * as path from "path";

const openAIFiles = new OpenAIFiles();
const filePath = path.resolve(process.cwd(), `../../../../../../examples/state_of_the_union.txt`);
const fileContents = await readFile(filePath);
const file = new File([fileContents], "state_of_the_union.txt", { type: "text/plain" });
await openAIFiles.createFile({
  file,
  purpose: "assistants",
});

{
  object: "file",
  id: "file-6gf6ZpEUHfsqVX6wovVLN7ES",
  purpose: "assistants",
  filename: "state_of_the_union.txt",
  bytes: 39027,
  created_at: 1703628349,
  status: "processed",
  status_details: null
}

If you run into an apiKey error, you can try to pass it directly as clientOptions:

In [13]:
const openAIFiles = new OpenAIFiles({clientOptions: { apiKey: process.env.OPENAI_API_KEY }});

## Use File in AI Assistant

In [27]:
import { OpenAIAssistantRunnable } from "langchain/experimental/openai_assistant";
import { OpenAIFiles } from "langchain/experimental/openai_files";
import * as path from "path";

const openAIFiles = new OpenAIFiles();
const filePath = path.resolve(process.cwd(), `../../../../../../examples/state_of_the_union.txt`);
const fileContents = await readFile(filePath);
const newFile = new File([fileContents], "state_of_the_union.txt", { type: "text/plain" });
const assistantFile = await openAIFiles.createFile({
  file: newFile,
  purpose: "assistants",
});

const agent = await OpenAIAssistantRunnable.createAssistant({
  model: "gpt-3.5-turbo-1106",
  instructions:
    "You are a weather bot. Use the provided functions to answer questions.",
  name: "Weather Assistant",
  tools: [...tools, { type: "retrieval" }],
  asAgent: true,
  fileIds: [assistantFile.id, fineTuneFile.id],
});

## Delete a File

Delete a file.

In [21]:
import { OpenAIFiles } from "langchain/experimental/openai_files";

const openAIFiles = new OpenAIFiles();
await openAIFiles.deleteFile({ fileId: file.id });

{ object: "file", id: "file-FuznXaLOGnYrXkSidit8n1Gu", deleted: true }

## List all Files

Returns a list of files that belong to the user's organization.

`purpose`?: string
Only return files with the given purpose.

In [22]:
import { OpenAIFiles } from "langchain/experimental/openai_files";

const openAIFiles = new OpenAIFiles();
await openAIFiles.listFiles({ purpose: "assistants" });

FileObjectsPage {
  options: { method: "get", path: "/files", query: {} },
  response: Response {
    body: ReadableStream { locked: true },
    bodyUsed: true,
    headers: Headers {
      "access-control-allow-origin": "*",
      "alt-svc": 'h3=":443"; ma=86400',
      "cf-cache-status": "DYNAMIC",
      "cf-ray": "83bc84d03dec82ed-IAD",
      "content-type": "application/json",
      date: "Tue, 26 Dec 2023 21:46:50 GMT",
      "openai-organization": "langchain",
      "openai-processing-ms": "118",
      "openai-version": "2020-10-01",
      server: "cloudflare",
      "set-cookie": "_cfuvid=I4.GDMaptjMvWz2HtjXa45Qp2C10t90AXPrYqJN1HxY-1703627210445-0-604800000; path=/; domain=.api.o"... 42 more characters,
      "strict-transport-security": "max-age=15724800; includeSubDomains",
      "x-request-id": "e62dc879943a0fdc6242e3c7b3656349"
    },
    ok: true,
    redirected: false,
    status: 200,
    statusText: "OK",
    url: "https://api.openai.com/v1/files"
  },
  body: {
    obje

## Retrieve File

Returns information about a specific file.

In [25]:
import { OpenAIFiles } from "langchain/experimental/openai_files";

const openAIFiles = new OpenAIFiles();
await openAIFiles.retrieveFile({ fileId: file.id });

{
  object: "file",
  id: "file-dBLRDhxODeEUeJKEpOtZqT5Q",
  purpose: "assistants",
  filename: "state_of_the_union.txt",
  bytes: 39027,
  created_at: 1703627254,
  status: "processed",
  status_details: null
}